In [2]:
from pyspark import SparkContext
sc = SparkContext()

In [3]:
import math

In [4]:
# carregar base de dados
import os.path
fileName = os.path.join('C:\spark\Data', 'DelayedFlights.csv')
numPartitions = 4
rawData = sc.textFile(fileName, numPartitions)


In [5]:
Cancelled = rawData.map(lambda x: x.split(",")[22])
featMonth = rawData.map(lambda x: x.split(",")[2])

In [161]:
# carregar base de dados
import os.path
fileName = os.path.join('C:\spark\Data', 'DelayedFlights.csv')

numPartitions = 4
rawData = sc.textFile(fileName, numPartitions)


from collections import Counter    #é o modo certo de fazer utilizando counter?


"""
def Entropia (classe):
    #counts = Counter(classe.collect())  #(tipo: qnt, tipo: qn) #não paralelizado
    
    counts = (classe.map(lambda x: (x, 1))
                   .reduceByKey(lambda a,b: a + b)
             )
    
    n = classe.count()  
    probs = counts.map(lambda x: x[1]/float(n))
    #probs = map( ..., counts.items()) #[c/float(sum(counts)) for _,c in counts]  #não paralelizado
    
    #entropia = -sum([p*math.log(p,2) for p in probs] )  #não paralelizado
    entropia = (probs.map(lambda p: -p*math.log(p,2))
                     .reduce(lambda a,b: a + b))

    return entropia
"""

def Entropia (classe):
    """Calcula a Entropia de Shannon de uma distribuição de dados.

    Args:
        classe (RDD): RDD contendo conjunto de dados a ser calculado a entropia.
                      Valores devem ser categóricos.

    Returns:
        float: (float): valor de Entropia de Shannon calculado para o RDD.
    """
    #counts calcula paralelamente o conteúdo do RDD
    #como tuplas contendo (tipo, quantidade)
    counts = (classe.map(lambda x: (x, 1))
                    .reduceByKey(lambda a,b: a + b))
    # n recebe o valor total de itens do RDD
    n = classe.count()
    
    # probs calcula a probabilidade de cada um dos estados do RDD
    probs = counts.map(lambda x: x[1]/float(n))
    
    # Entropia calcula a entropia do RDD
    ## a função map faz o calculo da Entropia de cada um dos estados
    ## a função reduce faz o somatório da entropia de Shannon
    entropia = (probs.map(lambda p: -p*math.log(p,2))
                     .reduce(lambda a,b: a + b))
    
    # retorna valor escalar referênte a entropia do RDD.
    return entropia

 
def infoGain (feature, classe, H):
    #feat_count = Counter(feature.collect())  #(tipo: qnt, tipo: qn) #não paralelizado
    feat_count = feature.map(lambda x: (x, 1))\
                        .reduceByKey(lambda a,b: a + b)\
                        .collect()
    #print feat_count
    
    #feature1 = feature.take(100)
    
    #teste = [f==v for v,c in feat_count for f in feature1] 
    #print teste
    
    #probsCondic = [classe.filter(feature.map(lambda x: x ==v).collect()).map(Entropia) for v,_ in feat_count]
    entropiasN = [Entropia(classe.zip(feature).filter(lambda x: x[1]==v).map(lambda x: x[0]))  for v,_ in feat_count]
    
    
    #return feat_count, entropiasN
    
    #probsCondic = [classe.filter(feature==v).map(Entropia).collect() for v in feat_count]

    #probsCondic = [classe.filter(feature==v).map(Entropia).collect() for v in feat_count]

    #probsCondic = [classe.filter(feature==v).map(Entropia).collect() for v,c in feat_count]
    

    
    #probs = probsCondic.map(Entropia)  #como filtrar classe através de um parametro no feature?
    
    
    #print probs
    #probs = [classe.filter(feature==v).map(Entropia) for v,c in feat_count.items()]  #como filtrar classe através de um parametro no feature?

    #print probs
    #n = sum([t[1] for t in feat_count])
    n = classe.count()
    #ig = H - sum([p/float(n)*p for p in entropiasN])
    
    
    ig = H - sum([(f[1]/n)*p for f,p in zip(feat_count, entropiasN)])

    
    print ig
    return ig
    

In [85]:
H =  Entropia (Cancelled)
print H

0.00425591582633


In [145]:
featSample = sc.parallelize(featMonth.takeSample(False, 1000))
classeSample = sc.parallelize(Cancelled.takeSample(False,1000))

In [146]:
featSample

ParallelCollectionRDD[619] at parallelize at PythonRDD.scala:175

In [162]:
ig = infoGain(featSample, classeSample, H)
print ig

0.00425591582633
0.00425591582633


In [152]:
print f, h

[(u'11', 68), (u'1', 91), (u'9', 48), (u'5', 77), (u'10', 50), (u'4', 81), (u'8', 71), (u'3', 99), (u'7', 98), (u'2', 108), (u'12', 115), (u'6', 94)] [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]


In [156]:
sum(dict(f).values())

1000

In [153]:
sum([p/float(sum(f))*p for p in h])

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [128]:
tmp = Cancelled.zip(featMonth).filter(lambda x: x[1]=='1').map(lambda x: x[0])

In [130]:
Entropia(tmp)

-0.0

In [131]:
print Counter(tmp.collect())  #(tipo: qnt, tipo: qn) #não paralelizado


Counter({u'0': 183527})


In [ ]:
#counts = Counter(classe.collect())  #(tipo: qnt, tipo: qn)
counts = Cancelled.map(lambda x: (x, 1))\
               .reduceByKey(lambda a,b: a + b)
    
print counts.take(2)

In [ ]:

counts = Counter(Cancelled.collect())  #(tipo: qnt, tipo: qn)
print counts

In [ ]:
Cancelled = rawData.map(lambda x: x.split(",")[22])
contagem = Cancelled.map(lambda x: (x, 1))\
                    .reduceByKey(lambda a,b: a + b)
print contagem.take(5)


In [ ]:
Cancelled = rawData.map(lambda x: x.split(",")[22])

featMonth = rawData.map(lambda x: x.split(",")[2])

#H =  Entropia (Cancelled)

In [ ]:
print infoGain (featMonth, Cancelled, H)

In [ ]:
teste = Counter(Cancelled.collect())


In [ ]:
print teste.values()

In [ ]:
sum(teste.values())

In [ ]:
[p*math.log(p) for _,p in test.item]

In [ ]:
print H


In [6]:
    # calcula paralelamente o conteúdo do RDD
    # como tuplas contendo (tipo, quantidade)
feat_count = Cancelled.map(lambda x: (x, 1))\
                        .reduceByKey(lambda a,b: a + b)\
                        .collect()
print feat_count

[(u'1', 633), (u'0', 1936125)]
